In [1]:
import tensorflow as tf
import numpy as np

In [2]:
char_arr = ['a', 'b', 'c', 'd', 'e', 'f', 'g',
            'h', 'i', 'j', 'k', 'l', 'm', 'n',
            'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']

In [3]:
char_dic = {n: i for i, n in enumerate(char_arr)}

In [4]:
words_arr = ['word', 'wood', 'deep', 'dive',
        'cold', 'cool', 'load', 'love',
        'kiss', 'kind']

In [5]:
def make_batch(data):
    input_batch, target_batch = [], []
    
    for word in data:
        idxs = [char_dic[i] for i in word[:-1]]
        target = char_dic[word[-1]]
        # make one-hot encoding
        input_batch.append(np.eye(len(char_dic))[idxs])
        target_batch.append(target)
        
        '''
        softmax_cross_entropy_with_logits takes label as one-hot, but
        sparse_softmax_cross_entropy_with_logits takes label as index.
        '''
        
    return input_batch, target_batch
        

In [6]:
lr = 1e-2
n_epochs = 30

n_steps = 3
n_inputs = n_classes = len(char_dic)

In [7]:
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
'''
Using sparse_softmax_cross_entropy_with_logits, x should be formed as
1-dimensional index numbers such as [1] [4] ..., not one-hot vectors.
When it takes one-hot vectors, x should take [None, n_classes].
'''
y = tf.placeholder(tf.int32, [None])

w = tf.Variable(tf.random_normal([128, n_classes]))
b = tf.Variable(tf.random_normal([n_classes]))

cell1 = tf.nn.rnn_cell.BasicLSTMCell(128)
cell1 = tf.nn.rnn_cell.DropoutWrapper(cell1, output_keep_prob=0.5)
cell2 = tf.nn.rnn_cell.BasicLSTMCell(128)

cells = tf.nn.rnn_cell.MultiRNNCell([cell1, cell2])

outputs, states = tf.nn.dynamic_rnn(cells, x, dtype=tf.float32)

outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]
output = tf.matmul(outputs, w) + b

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(\
                                        logits=output, labels=y))
optimizer = tf.train.AdamOptimizer(lr).minimize(loss)


Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [8]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, target_batch = make_batch(words_arr)

for epoch in range(n_epochs):
    _, loss = sess.run([optimizer, loss],
                      {x:input_batch, y:target_batch})
    print('epoch: {}, loss: {}'.format(epoch + 1, loss))
    
print('Complete!!!!!!!')

epoch: 1, loss: 3.9102890491485596


TypeError: Fetch argument 3.910289 has invalid type <class 'numpy.float32'>, must be a string or Tensor. (Can not convert a float32 into a Tensor or Operation.)